<a href="https://colab.research.google.com/github/morozowdmitry/semeval21/blob/master/semeval_toxic_spans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

import pickle

from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm

!git clone https://github.com/ipavlopoulos/toxic_spans.git
from toxic_spans.evaluation.semeval2021 import f1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Выделяем токены, для токенов размечаем, токсичные они или нет

In [ ]:
df = pd.read_csv('/content/tsd_trial.csv')
print(df.shape)
df.head()

In [ ]:
texts = df['text']
spans = df['spans']

for i in range(df.shape[0]):
  spans[i] = str(spans[i])
  if spans[i]!='[]':
    spans[i] = spans[i].replace('[','').replace(']','').replace('\'','').replace('"','').replace('\\','')
    spans[i] = spans[i].split(', ')
    spans[i] = [int(sp) for sp in spans[i]]
  else:
    spans[i] = []

print(spans[0])
print(type(spans[0]))

In [ ]:
values = []
values.append(['spans', 'text', 'tokens', 'is_toxic'])
for i, text in enumerate(texts):
  if i%1000 == 0:
    print(i)
  binstr = []
  tokens = []
  tmp=''
  for j in range(len(text)):
    if j==0:
      tmp+=text[j]
    else:
      if j in spans[i] and j-1 not in spans[i]:
        words = word_tokenize(tmp)
        for z in range(len(words)):
          tokens.append(words[z])
          binstr.append(0)
        tmp = text[j]
      elif j not in spans[i] and j-1 in spans[i]:
        words = word_tokenize(tmp)
        for z in range(len(words)):
          tokens.append(words[z])
          binstr.append(1)
        tmp = text[j]
      else:
        tmp+=text[j]
  if tmp!='':
    if len(text)-1 in spans[i]:
      words = word_tokenize(tmp)
      for z in range(len(words)):
        tokens.append(words[z])
        binstr.append(1)
    else:
      words = word_tokenize(tmp)
      for z in range(len(words)):
        tokens.append(words[z])
        binstr.append(0)
  #print(text)
  #print(spans[i])
  #print(binstr)
  #print(tokens)
  #print('******')
  values.append([spans[i], text, tokens, binstr])

pd.DataFrame(values).to_csv('tsd_trial_tokens.csv')

#Делаем обучающую выборку для токенов

In [ ]:
df = pd.read_csv('/content/tsd_trial_tokens.csv', index_col=0, header=1)
print(df.shape)
df.head()

In [ ]:
spans = df['spans']
texts = df['text']
tokens = df['tokens']
is_toxic = df['is_toxic']

for i in range(1, df.shape[0]):
  spans[i] = str(spans[i])
  if spans[i]!='[]':
    spans[i] = spans[i].replace('[','').replace(']','').replace('\'','').replace('"','').replace('\\','')
    spans[i] = spans[i].split(', ')
    spans[i] = [int(sp) for sp in spans[i]]
  else:
    spans[i] = []

for i in range(1, df.shape[0]):
  tokens[i] = str(tokens[i])
  if tokens[i]!='[]':
    tokens[i] = tokens[i].replace('[','').replace(']','').replace('\'','').replace('"','').replace('\\','')
    tokens[i] = tokens[i].split(', ')
    #tokens[i] = [int(sp) for sp in tokens[i]]
  else:
    tokens[i] = []

for i in range(1, df.shape[0]):
  is_toxic[i] = str(is_toxic[i])
  if is_toxic[i]!='[]':
    is_toxic[i] = is_toxic[i].replace('[','').replace(']','').replace('\'','').replace('"','').replace('\\','')
    is_toxic[i] = is_toxic[i].split(', ')
    is_toxic[i] = [int(sp) for sp in is_toxic[i]]
  else:
    is_toxic[i] = []

In [ ]:
train_sample = []
#train_sample.append(['is_toxic', 'token', 'token_lemmatized', 'text_id', 'text'])

num = 0

for i in range(1, spans.shape[0]):
  if i%1000==0:
    print(i)
  for j, token in enumerate(tokens[i]):
    if len(token)>=3:
      train_sample.append([is_toxic[i][j], token, wordnet_lemmatizer.lemmatize(token), i, texts[i]])

train_sample = pd.DataFrame(train_sample).sample(frac=1).values

train_sample_short = []
train_sample_short.append(['is_toxic', 'token', 'token_lemmatized', 'text_id', 'text'])


for tr in train_sample:
  if tr[0]==1:
    train_sample_short.append(tr)
  else:
    if num<=20856:
      train_sample_short.append(tr)
      num+=1
    
pd.DataFrame(train_sample_short).to_csv('tsd_train_fnn.csv')
print(len(train_sample_short))

In [ ]:
train_sample = []
train_sample.append(['is_toxic', 'token', 'token_lemmatized', 'text_id', 'text'])

num = 0

for i in range(1, spans.shape[0]):
  if i%1000==0:
    print(i)
  for j, token in enumerate(tokens[i]):
    if len(token)>=3:
      train_sample.append([is_toxic[i][j], token, wordnet_lemmatizer.lemmatize(token), i, texts[i]])
    
pd.DataFrame(train_sample).to_csv('tsd_trial_fnn.csv')
print(len(train_sample))

#Получаем эмбеддинги BERT для текстов

In [ ]:
df = pd.read_csv('/content/tsd_trial.csv')
print(df.shape)
df.head()

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
sentences = df['text'].values
sentence_embeddings = model.encode(sentences)

In [ ]:
print(sentences[0])
print(len(sentence_embeddings[0]))
print(sentence_embeddings[0])

In [ ]:
with open('/content/drive/My Drive/semeval21/sentence_embs_trial.pickle', 'wb') as f:
  pickle.dump(sentence_embeddings, f)

#Получаем эмбеддинги для слов

In [ ]:
df = pd.read_csv('/content/tsd_train_fnn.csv', index_col=0, header=1)
print(df.shape)
df.head()

words = set()
for el in df['token_lemmatized'].values:
  words.add(wordnet_lemmatizer.lemmatize(el.lower().replace('.','').replace(' ','')))

print(len(words))

In [ ]:
df = pd.read_csv('/content/tsd_trial_fnn.csv', index_col=0, header=1)
print(df.shape)
df.head()

for el in df['token_lemmatized'].values:
  words.add(wordnet_lemmatizer.lemmatize(el.lower().replace('.','').replace(' ','')))

print(len(words))

In [ ]:
fname = '/content/drive/My Drive/english_w2v/model.bin'
w2v = KeyedVectors.load_word2vec_format(fname, binary=True)  

In [ ]:
words = list(words)
print(words[:5])

embs = []
new_words = []

for w in words:
  try:
    embs.append(w2v[w])
    new_words.append(w)
  except:
    pass

print(len(embs))

In [ ]:
with open('/content/drive/My Drive/semeval21/word2vec_words.pickle', 'wb') as f:
  pickle.dump(new_words, f)
with open('/content/drive/My Drive/semeval21/word2vec_embs.pickle', 'wb') as f:
  pickle.dump(embs, f)

#Делаем обучающую выборку: эмбеддинг слова + эмбеддинг предложения

In [ ]:
df = pd.read_csv('/content/tsd_trial_fnn.csv', index_col=0, header =1)
df.head()

In [ ]:
list_words =[]
for el in df['token_lemmatized'].values:
  list_words.append(wordnet_lemmatizer.lemmatize(el.lower().replace('.','').replace(' ','')) )

print(len(list_words))

In [ ]:
with open('/content/drive/My Drive/semeval21/word2vec_words.pickle', 'rb') as f:
  words = pickle.load(f)
with open('/content/drive/My Drive/semeval21/word2vec_embs.pickle', 'rb') as f:
  embs = pickle.load(f)

with open('/content/drive/My Drive/semeval21/sentence_embs_trial.pickle', 'rb') as f:
  sentence_embs = pickle.load(f)

In [ ]:
train_data = []
train_labels = []

texts_words = []

words_set = set(words)

for i,el in enumerate(df.values):
  if list_words[i] in words_set:
    elem = np.concatenate((embs[words.index(list_words[i])], sentence_embs[el[3]-1]))
    train_labels.append(el[0])
    train_data.append(elem)
    texts_words.append([el[1],el[4]])

print(len(train_data))
print(len(train_labels))
print(train_data[0])
print(train_labels[0])

In [ ]:
with open('/content/drive/My Drive/semeval21/test_data.pickle', 'wb') as f:
  pickle.dump(train_data, f)
with open('/content/drive/My Drive/semeval21/test_labels.pickle', 'wb') as f:
  pickle.dump(train_labels, f)

In [ ]:
with open('/content/drive/My Drive/semeval21/train_key.pickle', 'wb') as f:
  pickle.dump(texts_words, f)

#Классификатор

In [ ]:
import keras
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Input, Embedding, Activation, Flatten, Dense, concatenate
from keras.models import Model

In [ ]:
with open('/content/drive/My Drive/semeval21/test_data.pickle', 'rb') as f:
  test_data = pickle.load(f)
with open('/content/drive/My Drive/semeval21/test_labels.pickle', 'rb') as f:
  test_labels = pickle.load(f)
with open('/content/drive/My Drive/semeval21/train_data.pickle', 'rb') as f:
  train_data = pickle.load(f)
with open('/content/drive/My Drive/semeval21/train_labels.pickle', 'rb') as f:
  train_labels = pickle.load(f)

In [ ]:
print(len(train_data))
print(len(train_labels))
print(len(test_data))
print(len(test_labels))

In [ ]:
#labels to categorical

train_labels = keras.utils.to_categorical(np.array(train_labels),2)
test_labels = keras.utils.to_categorical(np.array(test_labels),2)

In [ ]:
inputs=Input(shape=(len(train_data[0]),), name='input')
x=Dense(1024, activation='tanh', name='fully_connected_1024_tanh')(inputs)
#x=Dense(1024, activation='tanh', name='fully_connected_32')(x)
predictions=Dense(2, activation='softmax', name='output_softmax')(x)
model=Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

history = model.fit(np.array(train_data), train_labels, epochs=1, verbose=2,\
                    validation_data=(np.array(test_data), test_labels))
predict = np.argmax(model.predict(np.array(test_data)), axis=1)
answer = np.argmax(test_labels, axis=1)

f1=f1_score(predict, answer)*100
prec=precision_score(predict, answer)*100
recall=recall_score(predict, answer)*100
accuracy=accuracy_score(predict, answer)*100

print(i)
print('Готово!')
print('f1 = {}, accuracy = {}, precision = {}, recall = {}'.format(f1,accuracy,prec,recall))

In [ ]:
with open('/content/drive/My Drive/semeval21/test_key.pickle', 'rb') as f:
  test_key = pickle.load(f)
print(test_key[0])

In [ ]:
df = pd.read_csv('/content/tsd_trial.csv')

df.head()

In [ ]:
final_spans = []
for text in df['text'].values:
  spans = []
  for i, elem in enumerate(test_key):
    if elem[1]==text and predict[i]==1:
      pos = elem[1].find(elem[0])
      if pos>=0:
        for j in range(pos,pos+len(elem[0])):
          spans.append(j)
  final_spans.append(spans)

print(final_spans[:20])

In [ ]:
df = pd.read_csv('/content/tsd_trial.csv')


from ast import literal_eval
df.spans = df.spans.apply(literal_eval)

df['predictions'] = pd.Series(final_spans)
#df["f1_scores"] = df.apply(lambda row: f1(row.predictions, row.spans), axis=1)
df.head()

df.to_csv('/content/drive/My Drive/semeval21/res_4929_w2v_dist.csv')

In [ ]:
f1_scores = []

for i in range(690):
  f1_scores.append(f1(df['spans'][i],df['predictions'][i]))

np.mean(np.array(f1_scores))